In [ ]:
# %load init.py
import os
import pickle
import sys
# Enable module import from the parent directory from notebooks
sys.path.append(os.path.abspath('..'))
import time

import matplotlib as mpl
# Select plotting backend
mpl.use('nbAgg')

import matplotlib.pyplot as plt
# Customize plotting
plt.style.use('seaborn-paper')
plt.rcParams['axes.labelsize'] = 11.0
plt.rcParams['axes.titlesize'] = 12.0
plt.rcParams['errorbar.capsize'] = 3.0
plt.rcParams['figure.dpi'] = 72.0
plt.rcParams['figure.titlesize'] = 12.0
plt.rcParams['legend.fontsize'] = 10.
plt.rcParams['lines.linewidth'] = 1.
plt.rcParams['xtick.labelsize'] = 11.0
plt.rcParams['ytick.labelsize'] = 11.0

import numpy as np
import sympy as sp
sp.init_printing(euler=True, use_latex=True)

from IPython import display
from scipy import io, optimize
from sklearn import metrics

import core
import dynamicals
import kernels
import numericals
import utils

In [ ]:
dynamical = dynamicals.LotkaVolterra()

# Example

Run the inference algorithm

In [ ]:
config = core.Config()
config.create_time(0, 2, 100, 0, 2, 10, 0, 2, 10)
config.X_0 = np.array([5., 3.]) 
config.theta = np.array([2., 1., 1., 4.]) 
config.rho_2 = None
config.phi = [
    # (Kernal name, Kernal parameters)
    ('rbf', np.sqrt([2.5, 0.02])),
    ('rbf', np.sqrt([2.5, 0.02]))
]
config.sigma_2 = np.array([0.1, 0.1]) 
config.delta = np.full(dynamical.num_x, True)
config.gamma = np.array([5e-3, 5e-3]) 

config.opt_method = 'Newton-CG'
config.opt_tol = 1e-6
config.max_init_iter = None
config.max_iter = 2000

config.plotting_enabled = True
config.plotting_freq = 50

config.debug = False

config.spl_X = dynamical.generate_sample_path(config.theta, config.rho_2, config.X_0, config.spl_tps)
config.obs_Y = utils.collect_observations(config.spl_X, config.obs_t_indices, config.sigma_2)

gp = core.GaussianProcessRegression(dynamical, config)
gp.run()

lpmf = core.LaplaceMeanFieldODE(dynamical, config, gp)
lpmf.run()

# Experiments

Setup for the experiment

In [ ]:
num_repetitions = 10
directory = '../data/ode-lotka-volterra/{}/'
config_filename = utils.CONFIG_FILENAME
data_filename = utils.DATA_FILENAME
pos_data_filename = '{}-data-theta-pos.pickle'
vgmgp_directory = '/Users/ruifengxu/Development/ethz/gp_matching_for_benchmark_systems/results/Lotka_Volterra/{}/'
vgmgp_config_filename = 'lpmf.mat'

Run experiments

In [ ]:
for repetition in range(1, num_repetitions + 1):
    print('Starting repetition {}'.format(repetition))
    config = core.Config()
    config.load_config(directory.format(repetition), config_filename)
    gp = core.GaussianProcessRegression(dynamical, config)
    gp.run()
    lpmf = core.LaplaceMeanFieldODE(dynamical, config, gp)
    lpmf.run()
    lpmf.save_result(directory.format(repetition), data_filename.format(repetition))

Run experiments for positivity constraint on theta

In [ ]:
for repetition in range(1, num_repetitions + 1):
    print('Starting repetition {}'.format(repetition))
    config = core.Config()
    config.load_config(directory.format(repetition), config_filename)
    
    config.debug = True
    
    gp = core.GaussianProcessRegression(dynamical, config)
    gp.run()
    
    lpmf = core.LaplaceMeanFieldSymbolic(dynamical, config, gp)    
    lpmf.construct_theta_objective_functions(False, True)
    lpmf.construct_X_objective_functions(False, True)
    lpmf.run()
    lpmf.save_result(directory.format(repetition), pos_data_filename.format(repetition)) 

# Plotting

Load data for plotting

In [ ]:
lpmf_data = []
pos_data = []
vgmgp_data = []
for repetition in range(1, num_repetitions + 1):
    tmp = utils.load_data(directory.format(repetition), data_filename.format(repetition))
    lpmf_data.append(tmp)
    
    tmp = utils.load_data(directory.format(repetition), pos_data_filename.format(repetition))
    pos_data.append(tmp)
    
    with open(os.path.join(vgmgp_directory.format(repetition), vgmgp_config_filename), 'rb') as infile:
        tmp = io.loadmat(infile)    
        tmp['eta_X'] = tmp['est_X']
        tmp['eta_theta'] = tmp['est_theta'].ravel()
        tmp['eta_theta'][2],  tmp['eta_theta'][3] = tmp['eta_theta'][3],  tmp['eta_theta'][2]
        vgmgp_data.append(tmp)
        
config = core.Config()
config.load_config(directory.format(repetition), config_filename)   

Plot the state estimation result and the RMSE

In [ ]:
figure = plt.figure(figsize=(12, 4))

# State estimation
X_means = [
    utils.get_X_mean(lpmf_data), 
    utils.get_X_mean(pos_data), 
    utils.get_X_mean(vgmgp_data)
]
X_vars = [
    utils.get_X_var(lpmf_data), 
    utils.get_X_var(pos_data), 
    utils.get_X_var(vgmgp_data)
]
titles = ['LPMF', 'LPMF-POS', 'VGMGP']

for i in range(3):
    ax = figure.add_subplot(1, 4, i + 1)
    ax.plot(config.spl_tps, config.spl_X[0], color='C0', linewidth=1.5, label='Sample path')
    ax.scatter(config.obs_tps, config.obs_Y[0], color='C1', marker='x', label='Observation')
    ax.errorbar(config.est_tps, X_means[i][0], color='C2', linestyle='--', linewidth=1.5,
                yerr=np.sqrt(X_vars[i][0]), ecolor='0', elinewidth=1., capsize=3., capthick=.5, label='Estimation')
    
    ax.plot(config.spl_tps, config.spl_X[1], color='C0', linewidth=1.5)
    ax.scatter(config.obs_tps, config.obs_Y[1], color='C1', marker='x')
    ax.errorbar(config.est_tps, X_means[i][1], color='C2', linestyle='--', linewidth=1.5,
                yerr=np.sqrt(X_vars[i][1]), ecolor='0', elinewidth=1., capsize=3., capthick=.5)
    ax.set_xlabel('Time')
    ax.set_ylabel('State')
    ax.set_title(titles[i])
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles=handles, labels=labels, loc=0)    

# RMSE
boxprops = dict(linestyle='-', linewidth=1., color='0')
medianprops = dict(linestyle='-', linewidth=1.2, color='red')
meanpointprops = dict(marker='D', markersize=6., markeredgecolor='green', markerfacecolor='green')

lpmf_rmse = []
pos_rmse = []
vgmgp_rmse = []

for i in range(num_repetitions):
    lpmf_tmp = 0
    pos_tmp = 0
    vgmgp_tmp = 0
    for j in range(2):
        lpmf_tmp += metrics.mean_squared_error(lpmf_data[i]['eta_X'][j], vgmgp_data[i]['spl_X_true'][j])
        pos_tmp += metrics.mean_squared_error(pos_data[i]['eta_X'][j], vgmgp_data[i]['spl_X_true'][j])
        vgmgp_tmp += metrics.mean_squared_error(vgmgp_data[i]['eta_X'][j], vgmgp_data[i]['spl_X_true'][j])
    lpmf_rmse.append(0.5 * lpmf_tmp)
    pos_rmse.append(0.5 * pos_tmp)
    vgmgp_rmse.append(0.5 * vgmgp_tmp)
    
    
lpmf_rmse = np.sqrt(lpmf_rmse)
pos_rmse = np.sqrt(pos_rmse)
vgmgp_rmse = np.sqrt(vgmgp_rmse)

rmse_data = [
    lpmf_rmse,
    pos_rmse,
    vgmgp_rmse
]                      
ax = figure.add_subplot(1, 4, 4)
ax.boxplot(rmse_data, labels=titles, notch=False, showfliers=False, showmeans=True, 
           boxprops=boxprops, medianprops=medianprops, meanprops=meanpointprops, whis=[5, 95])
ax.set_ylabel('RMSE')
ax.set_xlabel('Method')

figure.tight_layout()
plt.show()
figure.savefig('lotka-states.eps', format='eps', dpi=1000, bbox_inches='tight')

Ploting parameter estimation

In [ ]:
figure = plt.figure(figsize=(8, 8))
for i in range(4):
    lpmf_theta = []
    pos_theta = []
    vgmgp_theta = []
    for j in range(num_repetitions):
        lpmf_theta.append(lpmf_data[j]['eta_theta'][i])
        pos_theta.append(pos_data[j]['eta_theta'][i])
        vgmgp_theta.append(vgmgp_data[j]['eta_theta'][i])
        
    ax = figure.add_subplot(2, 2, i + 1)
    theta_data = [
        lpmf_theta,
        pos_theta,
        vgmgp_theta
    ]
    ax.boxplot(theta_data, labels=titles, notch=False, showfliers=False, showmeans=True, 
               boxprops=boxprops, medianprops=medianprops, meanprops=meanpointprops, whis=[5, 95])
    ax.set_ylabel('$' + dynamical.theta_labels[i] + '$', fontsize=14)
    ax.plot(np.arange(6), np.full(6, config.theta[i]), linestyle='--', label='Truth')
    ax.set_xlabel('Method')
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles=handles, labels=labels, loc=0)     
    
figure.tight_layout()
plt.show()
figure.savefig('lotka-parameters-boxplot.eps', format='eps', dpi=1000, bbox_inches='tight')

Plot runtime

In [ ]:
lpmf_runtime = []
pos_runtime = []
vgmgp_runtime = []
for i in range(num_repetitions):
    lpmf_runtime.append(lpmf_data[i]['runtime'])
    pos_runtime.append(pos_data[i]['runtime'])
    vgmgp_runtime.append(vgmgp_data[i]['runtime'])
    
figure = plt.figure(figsize=plt.figaspect(1))
ax = plt.gca()
runtime_data = [
    lpmf_runtime,
    pos_runtime,
    vgmgp_runtime
]
ax.boxplot(runtime_data, labels=titles, notch=False, showfliers=False, showmeans=True, 
           boxprops=boxprops, medianprops=medianprops, meanprops=meanpointprops, whis=[5, 95])
ax.set_ylabel('Runtime (s)')
ax.set_xlabel('Method')
figure.tight_layout()
plt.show()    
figure.savefig('lotka-runtime-boxplot.eps', format='eps', dpi=1000, bbox_inches='tight')

In [ ]:
# Helper to transform the Lotka-Volterra experiment using VGMGP
config = core.Config()
config.create_time(0, 2, 100, 0, 2, 10, 0, 2, 10)
config.X_0 = np.array([5., 3.]) 
config.theta = np.array([2., 1., 1., 4.]) 
config.rho_2 = None

config.phi = [
    # (Kernal name, Kernal parameters)
    ('rbf', np.sqrt([2.5, 0.02])),
    ('rbf', np.sqrt([2.5, 0.02]))
]
config.sigma_2 = np.array([0.1, 0.1]) 
config.delta = np.full(2, True)
config.gamma = np.array([5e-3, 5e-3]) 

config.opt_method = 'Newton-CG'
config.opt_tol = 1e-6
config.max_init_iter = None
config.max_iter = 2000

config.plotting_enabled = False
config.plotting_freq = 50
config.plotting_config = None

directory = '../data/ode-lotka-volterra/{}/'

for repetition in range(1, num_repetitions + 1):
    with open(os.path.join(directory.format(repetition), 'lpmf.mat'), 'rb') as infile:
        config_mat = io.loadmat(infile)
    config.spl_X = config_mat['spl_X']
    config.obs_Y = config_mat['obs_Y']
    config.save_config(directory.format(repetition), config_filename)

In [ ]:
# Helper to check that the result are matching
for repetition in range(1, num_repetitions + 1):
    with open(os.path.join(directory.format(repetition), 'lpmf.mat'), 'rb') as infile:
        config_mat = io.loadmat(infile)    

    config_i = utils.load_data(directory.format(repetition), config_filename)
    
    assert np.alltrue(config_mat['spl_X'] == config_i['spl_X'])
    
    with open(os.path.join(vgmgp_directory.format(repetition), vgmgp_config_filename), 'rb') as infile:
        config_mat_original = io.loadmat(infile)
        
    assert np.alltrue(config_mat['spl_X'] == config_mat_original['spl_X'])
    
    for j in range(repetition, num_repetitions + 1):
        config_j = utils.load_data(directory.format(j), config_filename)
        if repetition == j:
            assert np.all(config_i['obs_Y'] == config_j['obs_Y'])
        else:
            assert np.any(config_i['obs_Y'] != config_j['obs_Y'])        